In [24]:
!pip install fastapi uvicorn pinecone sentence-transformers openai


In [25]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import openai

app = FastAPI()

PINECONE_API_KEY = "pcsk_2czr3A_58ustdWMomnjZ7p1P3P5UaNPagR9h61WNiW6iAeXFiDkH3gWrNoqpbxFghTZYCY"
INDEX_NAME = "bangla-book-index"
REGION = "us-east-1"
import os
OPENAI_API_KEY = ""

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
model = SentenceTransformer('intfloat/multilingual-e5-large')
openai.api_key = OPENAI_API_KEY

class QueryRequest(BaseModel):
    question: str

@app.post("/ask")
def ask_rag(request: QueryRequest):
    user_query = request.question

    # 1. Embed the query
    query_emb = model.encode(f"query: {user_query}")

    # 2. Retrieve top-K chunks
    K = 5
    results = index.query(
        vector=query_emb.tolist(),
        top_k=K,
        include_metadata=True
    )
    retrieved_chunks = [match['metadata']['text'] for match in results['matches']]
    context = "\n\n".join([f"[Passage {i+1}]\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])

    # 3. Build LLM prompt (copy from your notebook)
    llm_prompt = f"""
You are a Bangla QA assistant for textbooks. The question can be asked in both english or bangla but you need to answer in bangla.it in Bangla.

You have been provided with several retrieved passages from a textbook. These passages may include:
- Explanatory or narrative text (main passages)
- Tables
- MCQ (multiple choice questions) and their options or answer keys
- Lists and other formatted content

**Instructions:**
1. **If the question includes the word 'বাগধারাটি', first look for an answer in the word meaning section (such as sections titled 'বাগধারা', 'বাগধারার অর্থ', or where word meanings are explained).**
2. **First, carefully read the main explanatory/narrative passages. Try to find the answer to the user's question in these main text sections.**
3. **If you do not find the answer in the passages, then check the MCQ, option lists, tables, or answer keys in the retrieved text.**
4. If you still cannot find the answer, reply only with: 'Not found in the book.' Never make up information or use outside knowledge.

If multiple sources in the passages or MCQ part provide relevant details, synthesize the answer based on all of them.


Below are some examples:

User Question: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
Expected Answer: শুম্ভুনাথ

User Question: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
Expected Answer: মামাকে

User Question: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
Expected Answer: ১৫ বছর

Here are the most relevant passages:
{context}

Answer the following question using ONLY the information in the passages above.

Question: {user_query}
"""

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are an expert educational QA assistant."},
            {"role": "user", "content": llm_prompt}
        ],
        temperature=0.2,
        max_tokens=512
    )
    answer = response.choices[0].message.content

    return {"question": user_query, "answer": answer, "retrieved_chunks": retrieved_chunks}


In [ ]:
!ngrok config add-authtoken "30Q7jvgLPCHNVzoC2DLFpd1MG0g_3j5DvGQUg63QqhuCRmrGd"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run("my_fastapi_app:app", host="0.0.0.0", port=8000)

ERROR:    Error loading ASGI app. Could not import module "my_fastapi_app".
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/importer.py", line 19, in import_from_string
    module = importlib.import_module(module_str)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1140, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'my_fastapi_app'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/config.py", line 436, in load
    self.loaded_app = import_from_string(self.app)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

TypeError: object of type 'NoneType' has no len()